In [22]:
import yaml
from pathlib import Path
from utils import get_data_loaders, load_checkpoint, log_epoch, Criterion, add_cuts_to_config
import torch
from models import Model
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

In [19]:
cuda_id = 3
# log_name = '05_17_2022_22_08_41-GNN_full_dR_1'  # log id of the saved model to load
log_name = '05_17_2022_22_08_02-GNN_half_dR_1'  # log id of the saved model to load

setting = log_name.split('-')[1]

config = yaml.safe_load(Path(f'./configs/{setting}.yml').open('r'))
device = torch.device(f'cuda:{cuda_id}' if cuda_id >= 0 else 'cpu')
log_path = Path(config['data']['log_dir']) / log_name

if len(log_name.split('-')) > 2:
    cut_id = log_name.split('-')[2]
    config = add_cuts_to_config(config, cut_id)

In [20]:
data_loaders, x_dim, edge_attr_dim, dataset = get_data_loaders(setting, config['data'], config['optimizer']['batch_size'])

[Splits]
    train: 332406. # pos: 55401, # neg: 277005. Pos:Neg: 0.200
    valid: 71226. # pos: 11871, # neg: 59355. Pos:Neg: 0.200
    test: 674713. # pos: 11873, # neg: 662840. Pos:Neg: 0.018


In [15]:
df = pd.read_pickle(dataset.get_df_save_path())
len(df), len(dataset)

(578745, 1078345)

In [16]:
model = Model(x_dim, edge_attr_dim, config['data']['virtual_node'], config['model']).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=config['optimizer']['lr'])
load_checkpoint(model, optimizer, log_path, device)
criterion = Criterion(config['optimizer'])

[INFO] Loading checkpoint from 05_17_2022_22_08_02-GNN_half_dR_1


In [17]:
@torch.no_grad()
def eval_one_batch(data, model, criterion):
    model.eval()

    clf_logits = model(x=data.x, edge_index=data.edge_index, edge_attr=data.edge_attr, batch=data.batch, data=data)
    loss, loss_dict = criterion(clf_logits.sigmoid(), data.y)
    return loss_dict, clf_logits.data.cpu()


def run_one_epoch(data_loader, epoch, phase, device, model, criterion):
    loader_len = len(data_loader)
    run_one_batch = eval_one_batch
    phase = 'test ' if phase == 'test' else phase  # align tqdm desc bar

    all_loss_dict, all_clf_logits, all_clf_labels, all_sample_idx, all_endcap = {}, [], [], [], []
    pbar = tqdm(data_loader, total=loader_len)
    for idx, data in enumerate(pbar):
        loss_dict, clf_logits = run_one_batch(data.to(device), model, criterion)

        desc = log_epoch(epoch, phase, loss_dict, clf_logits, data.y.data.cpu(), batch=True)
        for k, v in loss_dict.items():
            all_loss_dict[k] = all_loss_dict.get(k, 0) + v
        all_clf_logits.append(clf_logits), all_clf_labels.append(data.y.data.cpu())
        all_sample_idx.append(data.sample_idx.data.cpu()), all_endcap.append(data.endcap.data.cpu())

        if idx == loader_len - 1:
            all_clf_logits, all_clf_labels = torch.cat(all_clf_logits), torch.cat(all_clf_labels)
            all_sample_idx, all_endcap = torch.cat(all_sample_idx), torch.cat(all_endcap)
            phase_idx = [phase.strip()] * len(all_sample_idx)
            for k, v in all_loss_dict.items():
                all_loss_dict[k] = v / loader_len
            desc, auroc, recall, avg_loss = log_epoch(epoch, phase, all_loss_dict, all_clf_logits, all_clf_labels, False, None)
        pbar.set_description(desc)

    return avg_loss, auroc, recall, all_clf_logits, all_sample_idx, all_endcap, phase_idx

In [18]:
clf_probs, all_sample_idx, all_endcap, all_phase_idx = [], [], [], []
for phase in ['train', 'valid', 'test']:
    avg_loss, auroc, recall, clf_logits, sample_idx, endcap, phase_idx = run_one_epoch(data_loaders[phase], 999, phase, device, model, criterion)
    clf_probs.append(clf_logits.sigmoid())
    all_sample_idx.append(sample_idx)
    all_endcap.append(endcap)
    all_phase_idx.extend(phase_idx)

clf_probs = torch.cat(clf_probs)
all_sample_idx = torch.cat(all_sample_idx)
all_endcap = torch.cat(all_endcap)
all_phase_idx = np.array(all_phase_idx)

[Epoch: 999]: train finished, focal: 0.002, total: 0.002, auroc: 0.982, recall@maxfpr: 0.652: 100%|██████████| 1299/1299 [00:44<00:00, 29.00it/s]
[Epoch: 999]: valid finished, focal: 0.003, total: 0.003, auroc: 0.930, recall@maxfpr: 0.469: 100%|██████████| 279/279 [00:10<00:00, 27.51it/s]
[Epoch: 999]: test  finished, focal: 0.002, total: 0.002, auroc: 0.934, recall@maxfpr: 0.469: 100%|██████████| 2636/2636 [01:34<00:00, 27.97it/s]


In [19]:
scores = pd.DataFrame({'sample_idx': all_sample_idx, 'probs': clf_probs.reshape(-1), 'endcap': all_endcap, 'phase': all_phase_idx})
scores['score_dict'] = scores.apply(lambda x: {x['endcap']: x['probs']}, axis=1)
scores['phase_dict'] = scores.apply(lambda x: {x['endcap']: x['phase']}, axis=1)
scores = scores.sort_values('sample_idx').reset_index(drop=True)

In [20]:
def agg_endcap(x):
    res = {}
    for each in x:
        res = res | each
    return res

In [21]:
agg_scores = scores.groupby('sample_idx')[['score_dict', 'phase_dict']].agg({'score_dict': agg_endcap, 'phase_dict': agg_endcap})
assert len(agg_scores) == len(df)

In [22]:
agg_scores.to_pickle(dataset.get_df_save_path().parent / f'{setting}-scores.pkl')

In [23]:
agg_scores

,score_dict,phase_dict
sample_idx,,
0,{-1: 0.713186502456665},{-1: 'train'}
1,{-1: 0.4296315610408783},{-1: 'train'}
2,{1: 0.3982222378253937},{1: 'test'}
3,{-1: 0.7315517067909241},{-1: 'train'}
4,{-1: 0.648301362991333},{-1: 'train'}
...,...,...
578740,"{1: 0.3504749536514282, -1: 0.4642656445503235}","{1: 'test', -1: 'test'}"
578741,"{-1: 0.42664122581481934, 1: 0.3977721929550171}","{-1: 'train', 1: 'test'}"
578742,"{1: 0.2887461185455322, -1: 0.46535834670066833}","{1: 'test', -1: 'test'}"


# Merge scores of different models

In [29]:
all_scores = os.listdir(dataset.get_df_save_path().parent)
all_scores = [dataset.get_df_save_path().parent / each for each in all_scores if 'GNN' in each]

In [30]:
merged_df = []
for score_file in all_scores:
    setting_name = score_file.name.split('-')[0]
    model_score = pd.read_pickle(score_file)
    model_score = model_score.rename(columns={'score_dict': f'{setting_name}-score_dict', 'phase_dict': f'{setting_name}-phase_dict'})
    merged_df.append(model_score)
merged_df = pd.concat(merged_df, axis=1)

In [31]:
merged_df.head()

,GNN_full_dR_1-score_dict,GNN_full_dR_1-phase_dict,GNN_half_dR_1-score_dict,GNN_half_dR_1-phase_dict
sample_idx,,,,
0,{0: 0.6919082999229431},{0: 'train'},{-1: 0.713186502456665},{-1: 'train'}
1,{0: 0.9665076732635498},{0: 'train'},{-1: 0.4296315610408783},{-1: 'train'}
2,{0: 0.74058997631073},{0: 'test'},{1: 0.3982222378253937},{1: 'test'}
3,{0: 0.8157235383987427},{0: 'train'},{-1: 0.7315517067909241},{-1: 'train'}
4,{0: 0.963468074798584},{0: 'train'},{-1: 0.648301362991333},{-1: 'train'}


In [21]:
merged_df.to_pickle(dataset.get_df_save_path().parent / f'merged-scores.pkl')